In [ ]:
!pip install wandb -Uq

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import wandb
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: simeonbetapudi (simeonbetapudi-belmont-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# check that there's more than just the CPU available
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")
if device==torch.device('cpu'): print("You should probably restart this with a GPU. It will be slow otherwise.")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Using device: cuda


In [4]:
# Available datasets with their normalization parameters
DATASETS = {
    'MNIST': {
        'dataset_class': torchvision.datasets.MNIST,
        'mean': (0.1307,), 'std': (0.3081,),
        'description': '28x28 grayscale handwritten digits (0-9)'
    },
    'FashionMNIST': {
        'dataset_class': torchvision.datasets.FashionMNIST,
        'mean': (0.2860,), 'std': (0.3530,),
        'description': '28x28 grayscale fashion items (10 classes: shirt, shoe, etc.)'
    },
    'CIFAR10': {
        'dataset_class': torchvision.datasets.CIFAR10,
        'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2023, 0.1994, 0.2010),
        'description': '32x32 color images (10 classes: planes, cars, etc.)'
    },
    'CIFAR100': {
        'dataset_class': torchvision.datasets.CIFAR100,
        'mean': (0.5071, 0.4867, 0.4408), 'std': (0.2675, 0.2565, 0.2761),
        'description': '32x32 color images (100 classes)'
    }
}

    #### YOUR CHOICES for dataset & activation functions
DATASET_NAME = 'CIFAR10'  # Choose between 'MNIST', 'FashionMNIST', 'CIFAR10', or 'CIFAR100'
BATCH_SIZE = 256 # could make this bigger

In [5]:
def get_dataset_info(dataset_name):
    """Get dataset configuration and sample to determine properties."""
    config = DATASETS[dataset_name]
    basic_transform = transforms.Compose([transforms.ToTensor()])
    temp_dataset = config['dataset_class'](root='./data', train=True, download=True, transform=basic_transform)
    sample_img, _ = temp_dataset[0]

    return {
        'config': config,
        'num_channels': sample_img.shape[0],
        'img_height': sample_img.shape[1],
        'img_width': sample_img.shape[2],
        'num_classes': len(temp_dataset.classes),
        'class_names': temp_dataset.classes
    }

dataset_info = get_dataset_info(DATASET_NAME)

print(f"\nDataset Properties:")
print(f"  Image size: {dataset_info['num_channels']}x{dataset_info['img_height']}x{dataset_info['img_width']}")
print(f"  Number of classes: {dataset_info['num_classes']}")
print(f"  Classes: {dataset_info['class_names'][:10]}...")  # Show first 10 classes

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(dataset_info['config']['mean'], dataset_info['config']['std'])
])

train_dataset = dataset_info['config']['dataset_class'](root='./data', train=True, download=False, transform=transform)
test_dataset = dataset_info['config']['dataset_class'](root='./data', train=False, download=False, transform=transform)


Dataset Properties:
  Image size: 3x32x32
  Number of classes: 10
  Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']...


In [6]:
class CNN(nn.Module):
    def __init__(self, dataset_info, activation=torch.nn.ReLU(), base_channels=32, channel_mult=2, n_conv_layers=4, kernel_size=3, stride=2):
        super(CNN, self).__init__()

        # Build conv layers dynamically
        self.conv_layers = nn.ModuleList()
        in_channels = dataset_info['num_channels']
        for i in range(n_conv_layers):
            out_channels = base_channels * (channel_mult ** i)
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=1))
            in_channels = out_channels

        self.activation = activation
        self.dropout1 = nn.Dropout(0.25)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) # put near end: yields one value per channel

        # Final channels after all conv layers
        final_channels = base_channels * (channel_mult ** (n_conv_layers - 1))
        self.fc = nn.Linear(final_channels, dataset_info['num_classes'])

    def forward(self, x):
        for conv_layer in self.conv_layers:
            x = self.activation(conv_layer(x))
            x = self.dropout1(x)
        x = self.global_avg_pool(x)  # one value per channel
        return self.fc(  x.flatten(start_dim=1) )  # flatten and run through linear layer



cnn_model = CNN(dataset_info).to(device)
print(f"CNN Model Parameters: {sum(p.numel() for p in cnn_model.parameters()):,}")

# quick test to make sure the forward() runs w/o errors:
rnd_batch = torch.randn([128,dataset_info['num_channels'], dataset_info['img_height'], dataset_info['img_width']]).to(device)
result = cnn_model(rnd_batch)
del rnd_batch

CNN Model Parameters: 390,986


In [7]:
def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    # Initialize wandb

    # Loss function:
    # Note: targets are just class indices (0-9), not one-hot vectors
    # nn.CrossEntropyLoss handles one-hot encoding internally for efficiency
    criterion = nn.CrossEntropyLoss()   # used for categorical variables, expects raw "logits"

    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # Use tqdm for progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')

        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct_train += pred.eq(target.view_as(pred)).sum().item()
            total_train += target.size(0)

            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct_train/total_train:.2f}%'
            })

        # Calculate epoch metrics
        epoch_loss = running_loss / len(train_loader)
        train_acc = 100. * correct_train / total_train

        # Evaluation phase
        model.eval()
        test_loss = 0
        correct_test = 0
        total_test = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct_test += pred.eq(target.view_as(pred)).sum().item()
                total_test += target.size(0)

        test_acc = 100. * correct_test / total_test

        # Store metrics
        train_losses.append(epoch_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_loss,
            "train_accuracy": train_acc,
            "test_accuracy": test_acc
        })

        print(f'Epoch {epoch+1}: Train Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%')

    wandb.finish()

    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
        'final_test_acc': test_acc
    }

In [8]:
# Expanded sweep config
sweep_config = {
    'method': 'bayes',   # options: grid, random, bayes
    'metric': {
        'name': 'test_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        # Learning rate search space
        # Activation function
        'activation': {
            'values': ['ReLU', 'LeakyReLU', 'GELU', 'SiLU']
        },

        'base_channels': {
            'values': [16, 32, 64, 128]
        },

        'channel_mult': {
            'values': [1, 2, 4]
        },

        'n_conv_layers': {
            'values': [3, 4, 5]
        },

        'kernel_size': {
            'values': [1, 3, 5, (3,5), (5,3)]
        },

        'stride': {
            'values': [1, 2, 3]
        }

    }
}

# Define available activation functions
activations = {
    "ReLU": torch.nn.ReLU(),
    "LeakyReLU": torch.nn.LeakyReLU(),
    "GELU": torch.nn.GELU(),
    "SiLU": torch.nn.SiLU()
}

In [ ]:
# Sweep-compatible training function
def sweep_train():

    wandb.init(project=f"convnet_{DATASET_NAME}_sweep", name=f"Idk Where this shows", reinit=True)
    config = wandb.config

    # Pick activation
    activation = activations[config.activation]

    # Build model
    model = CNN(dataset_info,
                activation=activation,
                base_channels=config.base_channels,
                channel_mult=config.channel_mult,
                n_conv_layers=config.n_conv_layers,
                kernel_size=config.kernel_size,
                stride=config.stride).to(device)


    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Train model
    results = train_model(model,
                          train_loader,
                          test_loader)

    return results


# Initialize and launch sweep
sweep_id = wandb.sweep(sweep_config, project="convnet_CIFAR10_sweep")
wandb.agent(sweep_id, function=sweep_train, count=10)  # run 10 experiments

Create sweep with ID: klsyko7c
Sweep URL: https://wandb.ai/simeonbetapudi-belmont-university/convnet_CIFAR10_sweep/sweeps/klsyko7c


wandb: Agent Starting Run: z686m9d9 with config:
wandb: 	activation: GELU
wandb: 	base_channels: 16
wandb: 	channel_mult: 1
wandb: 	kernel_size: 1
wandb: 	n_conv_layers: 3
wandb: 	stride: 3


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 1/10: 100%|██████████| 196/196 [00:12<00:00, 16.26it/s, Loss=2.3036, Acc=10.69%]


Epoch 1: Train Loss: 2.3037, Train Acc: 10.69%, Test Acc: 14.13%


...

Epoch 3/10: 100%|██████████| 196/196 [00:50<00:00,  3.87it/s, Loss=1.1289, Acc=60.00%]


Epoch 3: Train Loss: 1.1294, Train Acc: 60.00%, Test Acc: 62.86%


Epoch 4/10: 100%|██████████| 196/196 [00:50<00:00,  3.88it/s, Loss=1.0329, Acc=65.53%]


Epoch 4: Train Loss: 0.9880, Train Acc: 65.53%, Test Acc: 65.25%


Epoch 5/10:  21%|██▏       | 42/196 [00:10<00:39,  3.89it/s, Loss=0.9106, Acc=67.37%]